#### Combine Image

In [ ]:
import numpy as np
from skimage import io

In [ ]:
original_image_shape = (, , )

# Initialize an empty combined image
combined_image = np.zeros(original_image_shape, dtype=np.uint8)

# Coordinates of patches derived from bounding boxes
patch_coordinates = [
    (0, 0),     # Cluster 0
]

# Loop through each patch and place it in the combined image
for i, (y, x) in enumerate(patch_coordinates):
    patch = io.imread(f'./{i}.tif')
    y_end = min(y + patch.shape[0], combined_image.shape[0])
    x_end = min(x + patch.shape[1], combined_image.shape[1])
    patch_height = y_end - y
    patch_width = x_end - x
    combined_image[y:y_end, x:x_end, :] = patch[:patch_height, :patch_width, :]

# Save the combined image
io.imsave('Combined Image', combined_image)

#### Recunstruction

In [ ]:
import imageio
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Read images
image1 = imageio.imread('Registered Image').astype(float)
image2 = imageio.imread('Combine Image').astype(float)

# Ensure both images have the same number of channels
if image1.ndim != image2.ndim:
    raise ValueError("Images must have the same number of dimensions")

# Resize image2 to match image1's dimensions
height, width = image1.shape[:2]
image2 = cv2.resize(image2, (width, height))

# Convert images to 8-bit for thresholding, considering min-max normalization
image1_8bit = ((image1 - np.min(image1)) / (np.max(image1) - np.min(image1)) * 255).astype(np.uint8)
image2_8bit = ((image2 - np.min(image2)) / (np.max(image2) - np.min(image2)) * 255).astype(np.uint8)

# Apply adaptive thresholding for better background segmentation
threshold_type = cv2.THRESH_BINARY + cv2.THRESH_OTSU
_, mask1 = cv2.threshold(image1_8bit, 0, 255, threshold_type)
_, mask2 = cv2.threshold(image2_8bit, 0, 255, threshold_type)

# Convert masks to float for multiplication with original images
mask1 = mask1.astype(float) / 255.0
mask2 = mask2.astype(float) / 255.0

# Directly overlay image2 onto image1 using the mask
combined_image = (image1 * (1 - mask2)) + (image2 * mask2)

# Normalize the combined image to range [0, 255]
combined_image_normalized = 255 * (combined_image - np.min(combined_image)) / (np.max(combined_image) - np.min(combined_image))

# Apply Gaussian Blurring with a larger kernel for smoothing
smoothed_image = cv2.GaussianBlur(combined_image_normalized, (5,5), 0)

# Save and display the result
output_filename = "1-2-3-4-5.tif"
imageio.imwrite(output_filename, smoothed_image.astype(np.uint8))
print(f"Combined image saved as {output_filename}")

# Plotting for visualization
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(image1, cmap='gray')
ax[0].set_title("Original Image 1")
ax[1].imshow(image2, cmap='gray')
ax[1].set_title("Original Image 2")
ax[2].imshow(smoothed_image, cmap='gray')
ax[2].set_title("Combined Image")
for a in ax:
    a.axis('off')
plt.show()